# 09 Case Study

0. Introduction
1. Selecting Topics
    1. Choosing four Topics
    2. Process of Selecting Tweets 
    3. Presenting overview
    4. Topic 1
    5. Topic 2
    6. Topic 3
    7. Topic 4
2. LDA
3. BerTopic

In [146]:
from src.SampleTranslation05.translation_01 import load_samples
import pandas as pd
pd.options.display.max_columns = 30 # default 20
pd.options.display.max_rows = 60 # default 60
pd.options.display.float_format = '{:.2f}'.format
# pd.options.display.precision = 2
from src.explorativ_analysis_05 import split_strings_to_list

In [50]:
df_1 = pd.read_excel("samples_cleaned.xlsx")
df_1 = df_1.assign(
    translated_lemmas = df_1['translated_lemmas'].apply(split_strings_to_list),
    lemmas = df_1['lemmas'].apply(split_strings_to_list)
)
df_test = df_1[~df_1['topics'].isna()]
df_test = df_test[df_test['topics'].isin(['sanctions', 'unjustified_war', 'arms_delivery', 'people_killed'])]


In [49]:
df_test.columns

Index(['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'text', 'translated',
       'tweetcreatedts', 'tweetid', 'week', 'cleaned_text_translated',
       'cleaned_text', 'lemmas', 'adjs_verbs', 'nouns', 'entities', 'emojis',
       'translated_lemmas', 'translated_adjs_verbs', 'translated_nouns',
       'translated_entities', 'translated_emojis', 'lang', 'topics',
       'Unnamed: 21', 'topics_robin'],
      dtype='object')

In this case study, i will hand select 100 tweets and divide them into three topics. Then i will run LDA, BertTopic and BertTopic Multilingual on them and evaluate them against my handpicked topics.

In [4]:
from sentence_transformers import SentenceTransformer
model_labse = SentenceTransformer('sentence-transformers/LaBSE')

/Users/robinfeldmann/Library/Python/3.11/lib/python/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/robinfeldmann/Library/Python/3.11/lib/python/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [5]:
from src.TextEmbeddings08.text_embeddings import visualise_with_dimensional_reduction
from sklearn.decomposition import PCA
pca = PCA(n_components=2)

In [65]:
#df_test['embeddings_translated'] = df_test['cleaned_text_translated'].progress_apply(model_labse.encode)
#df_test['embeddings'] = df_test['cleaned_text'].progress_apply(model_labse.encode)
df_test['embeddings_lemmas'] = df_test['lemmas'].str.join(" ").progress_apply(model_labse.encode)

100%|██████████| 100/100 [00:05<00:00, 18.61it/s]


In [66]:
visualise_with_dimensional_reduction(df_test,PCA(n_components=2), color_to_show='topics', hover_data=['topics', 'cleaned_text_translated','lang'], row_to_transform="embeddings_lemmas")

In [150]:
visualise_with_dimensional_reduction(df_test,PCA(n_components=2), color_to_show='topics', hover_data=['topics', 'cleaned_text_translated','lang'])

In [131]:
visualise_with_dimensional_reduction(df_test,PCA(n_components=2), color_to_show='bertopic', hover_data=['topics', 'cleaned_text_translated','lang'])

In [149]:
for x in df_test.query('bertopic==2')['cleaned_text_translated']:
    print(x)

ukrainewar: the  eu  sanctions against  russia at a glance  ukrainerussiawar  ukrainerussiacrisis
the federal attorney general is investigating... possible  warcrimes  russia in  ukraine but not against president  putin personally. the international criminal court (icc) is also already investigating. after .../2
" ukraine needs heavy weapons," says ex-civil service worker, nikolas busse! 🤦😂
cnn: biden is finalizing sending patriot to ukraine  ukraina  russia  war
an american strategic reconnaissance drone rq-4 global hawk is circling off the coast of crimea. 😅🔥👌🏻  ukrainerussianwar  ukrainecounteroffensive  war on ukraine  ukraine️  russiaukrainewar  russiaisaterroriststate  usa
more and more evidence of  russian war crimes uncovered on a near daily basis in  ukraine️. all being documented and sent to the  hague.
twenty-two individuals and two entities are subject to new anti-russian sanctions by the u.s. treasury department's office of foreign assets control (ofac).  usa  russia  sanc

In [58]:
len(word2id.items())

972

In [141]:
from gensim.corpora.dictionary import Dictionary
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"


topics = df_test.groupby("topics")["lemmas"].sum()

word2id = Dictionary( df_test['translated_lemmas'].to_list())

In [143]:

cm = CoherenceModel(topics=topics, 
                    texts= df_test['lemmas'].to_list(),
                    coherence='c_npmi',  
                    dictionary=word2id)

coherence_per_topic = cm.get_coherence_per_topic()

coherence_per_topic

[-0.4559086739112057,
 -0.020213858440038025,
 -0.5163117696406291,
 0.11488970981405285]

[0.050937533124918506,
 0.0808065847719886,
 -0.06441845094015212,
 0.058774471751185525]

In [83]:
df_test['translated_lemmas'].shape

(100,)

In [80]:
import numpy as np
np.stack(df_test['embeddings'].to_list(), axis=0).shape

(100, 768)

In [127]:
from bertopic import BERTopic
topic_model = BERTopic()
topics, probs = topic_model.fit_transform(df_test['cleaned_text'].to_list(), np.stack(df_test['embeddings'].to_list(), axis=0))

In [128]:
topic_model.get_topic_info()['Representation'].to_list()

[['la', 'de', 'un', 'ukraine', 'di', 'in', 'le', 'guerre', 'hanno', 'putin'],
 ['der', 'russia', 'the', 'di', 'la', 'ukraine', 'gas', 'nato', 'putin', 'to'],
 ['ukraine', 'de', 'tus', 'dans', 'ont', 'en', 'les', 'moins', 'au', 'blesss'],
 ['de',
  'ukraine',
  'russe',
  'guerre',
  'la',
  'le',
  'poutine',
  'et',
  'vladimir',
  'crimes']]

In [129]:
df_test['bertopic'] = topics

In [130]:
cm = CoherenceModel(topics=df_test.groupby("bertopic")["lemmas"].sum(),  texts= df_test['lemmas'].to_list(),
                    coherence='c_npmi',  
                    dictionary=word2id)


coherence_per_topic = cm.get_coherence_per_topic()

coherence_per_topic

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

[-0.3510333663279367,
 -0.5132066730551073,
 -0.030135892566192587,
 -0.007000228560202858]